## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [21]:
# download the data:
!wget "https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1" -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2022-09-08 10:36:25--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2022-09-08 10:36:27--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1dca44c9ff35d06917998e3777.dl.dropboxusercontent.com/cd/0/get/BsjOvXq33EWpevNhYPNGNAsUqmepGIUdnC_UY-znUB_qeZpVoBKDh4lW4bsOxC4KVW0nB3mnxG_hTW_cVWJ3h-hOW1tGBxqx_VfJc1_ymigxBjqv38Bn0MNnM5Qtz_FdweDuST7IXPF92IpC5fSN5-Tx-FxTFTpjt9HpMS0tNmbFJg/file?dl=1# [following]
--2022-09-08 10:36:27--  https://uc1dca44c9ff35d06917998e3777.dl.dropboxusercontent.com/cd/0/get/BsjOvXq33EWpevNhYPNGNAsUq

In [22]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [23]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [24]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(x.lower()) for x in data]

In [25]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [26]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [27]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,         # consider words that occured at least 5 times
                 window=5).wv         # define context as a 5-word window around the target word

In [28]:
# now you can get word vectors !
model.get_vector('anything')

array([-4.1257854 ,  0.1984146 , -0.09978543,  1.073815  ,  3.7490122 ,
        0.06100158,  2.350467  , -3.101195  , -0.5255953 ,  3.0150807 ,
       -0.00882237,  2.6476862 ,  3.7971318 ,  0.3897424 ,  1.2247715 ,
       -1.6433554 , -1.749886  , -0.7007468 ,  1.5450927 , -0.97985053,
       -2.932081  , -1.5873622 ,  0.00705901, -1.2568667 ,  0.16641082,
       -2.8749127 ,  1.6572757 , -1.3748478 , -0.11457453,  1.7385174 ,
       -0.99025756, -1.6503323 ], dtype=float32)

In [29]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9485090970993042),
 ('sauce', 0.9339588284492493),
 ('butter', 0.9230124354362488),
 ('cheese', 0.9209069013595581),
 ('potato', 0.9175829291343689),
 ('fruit', 0.9146989583969116),
 ('wine', 0.906436562538147),
 ('chocolate', 0.904493510723114),
 ('honey', 0.8968783020973206),
 ('pasta', 0.8966920971870422)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [30]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [31]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.4922019839286804),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [32]:
words = sorted(model.key_to_index, 
               key=lambda word: model.get_vecattr(word, "count"),
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [57]:
# for each word, compute it's vector with model

word_vectors = model[words]

In [58]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [64]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
pca = PCA(n_components=2)
word_vectors_pca = pca.fit_transform(word_vectors)

# after that, normalize vectors to make sure they have zero mean and unit variance
scaler = StandardScaler()
word_vectors_pca = scaler.fit_transform(word_vectors_pca)

In [65]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [66]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [67]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1004', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [71]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca

tsne = TSNE()
word_tsne = tsne.fit_transform(word_vectors)

word_tsne = scaler.fit_transform(word_tsne)

/home/darinka/.local/share/virtualenvs/nlp_course-oo_HWYIR/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/darinka/.local/share/virtualenvs/nlp_course-oo_HWYIR/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [72]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1114', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [76]:
np.average([[2, 3], [2, 4]], axis=0)

array([2. , 3.5])

In [81]:
def get_phrase_embedding(phrase: str):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    phrase_tok = tokenizer.tokenize(phrase.lower())

    # it would work, if it was guaranteed, that all the words
    # the phrase are in the model's vocabulary
    # vector = np.average(model[phrase_tok], axis=0)

    # however... we need to "clean" up the phrase
    valid_phrase = [word for word in phrase_tok if word in model]
    
    if valid_phrase:
        vector = np.average(model[valid_phrase], axis=0)
    
    return vector
        
    

In [80]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [82]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [83]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [84]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

/home/darinka/.local/share/virtualenvs/nlp_course-oo_HWYIR/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/darinka/.local/share/virtualenvs/nlp_course-oo_HWYIR/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [87]:
# find the synonyms of the phrase
query = get_phrase_embedding("How do I enter the matrix?")

In [101]:
cosines  = [vec @ query / np.linalg.norm(vec) / np.linalg.norm(query) for vec in phrase_vectors]

In [91]:
for i in np.argsort(cosines)[-5:]:
    print(chosen_phrases[i])

How do I run a shell script from Java code?

My WhatsApp chat backup got deleted from Google, I need to switch from one Android to another, the chat is there only on the phone. What should I do?

How do I listen a song from you?

If I wanted to learn about the Roman Empire,what would be the best books to read?

How do I learn Calculus on my own?



In [85]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1235', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [92]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [167]:
from sklearn.metrics.pairwise import cosine_distances

In [176]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query_emb = get_phrase_embedding(query)
    cosines = np.array([cosine_distances([query_emb], [vec]) for vec in data_vectors]).squeeze()

    return [data[i] for i in np.argpartition(cosines, k)[:k]]

In [177]:
results = find_nearest(query="How do i enter the matrix?", k=10)
print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
How do I use the Greenify app?
What should I do to enter hollywood?
What can I do to save the world?
How do I win this?
How do I use the pad in MMA?
How do I think out of the box? How do I learn to think out of the box?
What do I do to enter the line of event management?
How do I find the 5th dimension?
How do I estimate the competition?



In [178]:
find_nearest(query="How does Trump?", k=10)

['What does India think of Donald Trump?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think of India?\n',
 'What does Donald Trump think about Israel?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [179]:
find_nearest(query="Why don't i ask a question myself?", k=10)

["Why don't I get a girlfriend?\n",
 "Why don't I ever get a girl?\n",
 "Why can't I take a joke?\n",
 "Can I ask a girl out that I don't know?\n",
 "I don't have no question?\n",
 "Why don't I have a boyfriend?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why don't I have a girlfriend?\n"]

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.